In [ ]:
import math
import os
import pandas as pd
import networkx as nx
import numpy as np
import time
import openpyxl
from openpyxl import Workbook
import random
from scipy.stats import kendalltau
import igraph as ig
import leidenalg as la

def measure_execution_time(method, G):
    start_time = time.time()
    result = method(G)
    ranked_nodes = sorted(result.items(), key=lambda x: x[1], reverse=True)
    end_time = time.time()
    execution_time = end_time - start_time
    return ranked_nodes, execution_time

def compute_monotonicity(G, ranked_nodes):
    ranks = [score for node, score in ranked_nodes]
    unique_ranks = list(set(ranks))
    n = G.number_of_nodes()
    nr_dict = {rank: ranks.count(rank) for rank in unique_ranks}
    nr_sum = sum(nr * (nr - 1) for nr in nr_dict.values())
    M = (1 - nr_sum / (n * (n - 1))) ** 2
    return M

def compute_mrr(ground_truth, ranked_list, top_ratios):
    mrr_scores = {}
    num_nodes = len(ground_truth)
    
    for ratio in top_ratios:
        k = max(1, int(ratio * num_nodes))
        reciprocal_ranks = []
        
        for node in ground_truth[:k]:
            if node in ranked_list:
                rank = ranked_list.index(node) + 1
                reciprocal_ranks.append(1 / rank)
        
        mrr_scores[f'MRR {ratio:.2f}'] = sum(reciprocal_ranks) / k if reciprocal_ranks else 0
    
    return mrr_scores


def compute_rbo(sigma, R, alpha=0.9):
    """Compute the Rank-Biased Overlap (RBO) between two ranking lists."""
    def A(sigma, R, f):
        sigma_f = set(sigma[:f])
        R_f = set(R[:f])
        if len(sigma_f) == 0 and len(R_f) == 0:
            return 1
        return len(sigma_f & R_f) / len(sigma_f | R_f)

    n = max(len(sigma), len(R))
    rbo = (1 - alpha) * sum(alpha**(f - 1) * A(sigma, R, f) for f in range(1, n + 1))
    return rbo

def compute_jaccard_similarity(set1, set2):
    return len(set1 & set2) / len(set1 | set2) if set1 | set2 else 0

# Define methods
# method1: Degree
def method1(G):
    return nx.degree_centrality(G)

# method2: Betweenness Centrality
def method2(G):
    return nx.betweenness_centrality(G)

# method3: Closeness Centrality
def method3(G):
    return nx.closeness_centrality(G)

# method4: k-shell
def method4(G):
    return nx.core_number(G)

def CnC(G):
    ks = nx.core_number(G)
    cnc = {vi: sum(ks[vj] for vj in G.neighbors(vi)) for vi in G.nodes()}
    return cnc

# method5: CncPlus
def method5(G):
    ks = nx.core_number(G)
    cnc = CnC(G)
    cnc_plus = {vi: sum(cnc[vj] for vj in G.neighbors(vi)) for vi in G.nodes()}
    return cnc_plus

def HIndex(G):
    h_index = {}
    for node in G.nodes():
        neighbors = list(G.neighbors(node))
        degrees = sorted([G.degree(neighbor) for neighbor in neighbors], reverse=True)
        h = 0
        for i, degree in enumerate(degrees):
            if degree >= i + 1:
                h = i + 1
            else:
                break
        h_index[node] = h
    return h_index

# method6: Local H-Index
def method6(G):
    HI = HIndex(G)
    LHI = {vi: HI[vi] + sum(HI[vj] for vj in G.neighbors(vi)) for vi in G.nodes()}
    return LHI

# method7: DNC
def method7(G):
    SLC = {}
    k = dict(nx.degree(G))
    C = nx.clustering(G)
    for i in G.nodes():
        SLC[i] = sum(C[j] for j in G.neighbors(i))
    
    DNC = {}
    alpha = 1
    for i in G.nodes():
        DNC[i] = k[i] + alpha * SLC[i]
    return DNC

#method8:ECLGC 
# Method 8: ECLGC
def method8(G):
    # Precompute clustering coefficients
    CLC = nx.clustering(G)
    
    # Precompute neighbors for each node
    neighbors = {node: list(G.neighbors(node)) for node in G.nodes()}
    
    # Precompute shortest path lengths for all node pairs
    sp_lengths = dict(nx.all_pairs_shortest_path_length(G))
    
    # Compute CLGC (not used later, but computed as in the original code)
    CLGC = {}
    for i in G.nodes():
        s = 0.0
        for j in G.nodes():
            if i == j:
                continue
            if j in sp_lengths[i]:
                s += math.sqrt(CLC[j]) / sp_lengths[i][j]
        CLGC[i] = s * CLC[i]
    
    # Compute ECLGC
    ECLGC = {}
    for v in G.nodes():
        sum1 = 0.0
        sum3 = 0.0
        neighbors_v = neighbors[v]
        if not neighbors_v:  # if no neighbors, ECLGC is 0
            ECLGC[v] = 0
            continue
        
        for u in G.nodes():
            if u == v:
                continue
            if u in sp_lengths[v]:
                # Precompute sum2 for u's neighbors
                neighbors_u = neighbors[u]
                lu = len(neighbors_u) if neighbors_u else 1  # avoid division by zero
                sum2 = 0.0
                for j in neighbors_u:
                    sum2 += CLC[j] / lu
                
                sum1 += CLC[u] / len(neighbors_v)
                d = sp_lengths[v][u]  # shortest path length from v to u
                sum3 += math.sqrt(CLC[u] + sum2) / d
        ECLGC[v] = sum1 * sum3
    return ECLGC


def get_neighborhood(G, node, radius):
    neighborhood = nx.single_source_shortest_path_length(G, node, cutoff=radius)
    neighborhood.pop(node, None)
    return list(neighborhood.keys())


# method9: CenC
def compute_alpha(graph):
    size_of_V = graph.number_of_nodes()
    # Ensure order_of_magnitude is at least 1 to avoid division by zero.
    order_of_magnitude = max(int(math.log10(size_of_V)), 1)
    alpha = size_of_V / order_of_magnitude
    return alpha

def method9(G, m=1):
    alpha = compute_alpha(G)
    # Cap alpha to avoid math range error in math.exp (overflow for very large alpha)
    capped_alpha = min(alpha, 700)
    
    degree_centrality = nx.degree_centrality(G)
    clustering_coeff = nx.clustering(G)
    kshell = nx.core_number(G)
    
    centrality = {}
    
    for i in G.nodes():
        cenC_i = 0
        
        # Precompute the m-neighborhood and distances from i
        sp_lengths_i = nx.single_source_shortest_path_length(G, i, cutoff=m)
        m_neighborhood = set(sp_lengths_i.keys())
        m_neighborhood.discard(i)  # Exclude the node itself
        
        for j in m_neighborhood:
            d_ij = sp_lengths_i[j]  # Use precomputed distance
            if d_ij == 0:
                continue
            
            DC_i = degree_centrality[i]
            C_i = clustering_coeff[i]
            k_s_i = kshell[i]
            k_s_j = kshell[j]
            
            # Compute term1 with a safe exponentiation
            term1 = (math.exp(capped_alpha) * DC_i + (1 / C_i if C_i != 0 else 0))
            term2 = k_s_j / (abs(k_s_i - k_s_j) + 1)
            
            cenC_i += (4 * math.pi**2) * (term1 / (d_ij**2)) * term2
        
        centrality[i] = cenC_i
    
    return centrality

def k_shell(graph):
    return nx.core_number(graph)

def evaluate_power_k_shell(k_si, k_sj):
    return math.sqrt(k_si + k_sj)

def compute_P_ji(graph, node_i, node_j):
    try:
        a_ji = graph[node_j][node_i]['weight'] if 'weight' in graph[node_j][node_i] else 1
        total_a_ji = sum(graph[node_j][nbr]['weight'] if 'weight' in graph[node_j][nbr] else 1 for nbr in graph.neighbors(node_j))
        return a_ji / total_a_ji
    except KeyError:
        return 0

def evaluate_ED(graph, node_i, node_j):
    P_ji = compute_P_ji(graph, node_i, node_j)
    if P_ji == 0:
        return 1  # Prevent log(0)
    return max(1 - math.log(P_ji), 1)

def evaluate_influence(graph, node_i, node_j, k_shell_values):
    alpha_ij = evaluate_power_k_shell(k_shell_values[node_i], k_shell_values[node_j])
    ED_ij = evaluate_ED(graph, node_i, node_j)
    degree_vi = graph.degree(node_i)
    return alpha_ij * degree_vi / ED_ij

# EDBC
def method10(G):
    k_shell_values = k_shell(G)
    EDBC = {u: 0 for u in G.nodes()}
    
    for u in G.nodes():
        for v in G.neighbors(u):
            for neighbor_u in G.neighbors(u):
                EDBC[v] += evaluate_influence(G, v, neighbor_u, k_shell_values)
    
    return EDBC

# Method 11 (LGC)
def method11(G):
    LC = {}
    K = dict(nx.degree(G))
    for i in G.nodes():
        LC[i] = sum(K[j] for j in nx.neighbors(G, i)) * 2 + K[i] ** 2 + K[i]

    LGC = {}
    radius = 3
    for vi in G.nodes():
        LGC[vi] = 0
        neighbors = get_neighborhood(G, vi, radius)
        for vj in neighbors:
            try:
                d_ij = nx.shortest_path_length(G, vi, vj)
                if d_ij != 0:
                    LGC[vi] += LC[vi] * LC[vj] / d_ij ** 2
            except nx.NetworkXNoPath:
                continue

    return LGC

# Method 12 (SEGM)
def method12(G):
    k = dict(G.degree())
    I = dict()
    E = dict()
    SE = dict()

    for node in G.nodes:
        I[node] = 0

    for i in G.nodes:
        sum_kj = 0
        for j in G.neighbors(i):
            sum_kj += k[j]
        if sum_kj != 0:
            I[i] = k[i] / sum_kj
        sum_e = 0
        for j in G.neighbors(i):
            if I[j] != 0:
                sum_e += I[j] * np.log(I[j])
        E[i] = -1 * sum_e
        SE[i] = np.exp(E[i]) * k[i]

    SEGM = dict()
    R = 3
    for i in G.nodes:
        SEGM[i] = 0
        neighbors = get_neighborhood(G, i, R)
        for j in neighbors:
            if nx.shortest_path_length(G, i, j) != 0:
                SEGM[i] += (SE[i] * SE[j]) / (nx.shortest_path_length(G, i, j) ** 2)

    return SEGM

# Method 13 (MCGM)
def method13(G):
    # Compute degree and core number
    k = dict(G.degree())
    ks = nx.core_number(G)
    
    # Compute eigenvector centrality with increased iterations.
    try:
        x = nx.eigenvector_centrality(G, max_iter=500, tol=1e-06)
    except nx.PowerIterationFailedConvergence:
        # Fallback to NumPy-based eigenvector centrality if convergence fails.
        x = nx.eigenvector_centrality_numpy(G)
    
    # Convert centrality measures to arrays for median and max computations.
    k_values = np.array(list(k.values()))
    ks_values = np.array(list(ks.values()))
    x_values = np.array(list(x.values()))
    
    # Compute medians and maximum values.
    k_mid = np.median(k_values)
    ks_mid = np.median(ks_values)
    x_mid = np.median(x_values)
    
    k_max = np.max(k_values)
    ks_max = np.max(ks_values)
    x_max = np.max(x_values)
    
    # Calculate alpha, ensuring no division by zero.
    if ks_mid == 0:
        alpha = 0
    else:
        alpha = max(k_mid / k_max, x_mid / x_max) / (ks_mid / ks_max)
    
    MCGM = {}
    R = 3  # Radius for considering node pairs.
    
    # Precompute shortest path lengths for all nodes with a cutoff of R.
    sp_lengths = dict(nx.all_pairs_shortest_path_length(G, cutoff=R))
    
    for i in G.nodes():
        MCGM[i] = 0
        # Iterate over nodes reachable from i within cutoff R.
        for j, d in sp_lengths.get(i, {}).items():
            if i == j or d == 0:
                continue
            # Only consider nodes within radius R.
            if d <= R:
                # Compute contribution from node j.
                term_i = (k[i] / k_max + alpha * ks[i] / ks_max + x[i] / x_max)
                term_j = (k[j] / k_max + alpha * ks[j] / ks_max + x[j] / x_max)
                MCGM[i] += term_i * term_j / (d ** 2)
    
    return MCGM

# Method 14 (HVGC)
def compute_H_index(G, i):
    degrees = [G.degree(neighbor) for neighbor in G.neighbors(i)]
    degrees.sort(reverse=True)
    H_index = 0
    for idx, degree in enumerate(degrees):
        if degree >= idx + 1:
            H_index = idx + 1
        else:
            break
    return H_index

def compute_H_v(G, i):
    H_i = compute_H_index(G, i)
    H_v_i = sum(G.degree(j) for j in G.neighbors(i) if G.degree(j) >= H_i)
    return H_v_i

def compute_c_i(G, i):
    def compute_p_ij(G, i, j):
        neighbors_i = list(G.neighbors(i))
        sum_z_iw = sum(G[i][w].get('weight', 1) for w in neighbors_i)
        p_ij = G[i][j].get('weight', 1) / sum_z_iw if sum_z_iw != 0 else 0
        return p_ij

    c_i = 0
    neighbors_i = list(G.neighbors(i))
    for j in neighbors_i:
        p_ij = compute_p_ij(G, i, j)
        neighbors_j = list(G.neighbors(j))
        common_neighbors = set(neighbors_i).intersection(neighbors_j)
        sum_piw_pwj = sum(compute_p_ij(G, i, w) * compute_p_ij(G, w, j) for w in common_neighbors)
        term = p_ij + sum_piw_pwj
        c_i += term ** 2
    
    return c_i

def compute_HVGC(G, i, H_v, R):
    c_i = compute_c_i(G, i)
    HVGC_i = 0
    for j in G.nodes():
        if j != i:
            d_ij = nx.shortest_path_length(G, source=i, target=j)
            if d_ij <= R:
                HVGC_i += (math.exp(-c_i) * H_v[i] * H_v[j]) / (d_ij ** 2)
    return HVGC_i

#HVGC
def method14(G):
    H_v = {i: compute_H_v(G, i) for i in G.nodes()}
    R = 2
    HVGC = {node: compute_HVGC(G, node, H_v, R) for node in G.nodes()}
    return HVGC


# Method15 (BaseGM)
def method15(G):
    def get_neighborhood(G, node, radius):
        neighborhood = nx.single_source_shortest_path_length(G, node, cutoff=radius)
        neighborhood.pop(node, None)
        return list(neighborhood.keys())

    gravity = dict()
    kshell = nx.core_number(G)
    for vi in G.nodes():
        gravity[vi] = 0
        neighbors = get_neighborhood(G, vi, 3)
        for vj in neighbors:
            gravity[vi] += ((kshell[vi] * kshell[vj]) / nx.shortest_path_length(G, vi, vj) ** 2)

    gravity_plus = dict()
    for vi in G.nodes():
        gravity_plus[vi] = 0
        for vj in G.neighbors(vi):
            gravity_plus[vi] += gravity[vj]

    return gravity_plus

# Method 16 (HKS)
def method16(G):
    def set_b_values(G):
        b, Shell, b_values = 1, 1, {}
        while G.number_of_nodes() > 0:
            flag = False
            for v in list(G.nodes):
                if G.degree[v] <= Shell:
                    b_values[v] = b
                    flag = True
            
            if flag:
                G.remove_nodes_from([v for v in list(G.nodes) if b_values.get(v) == b])
                b += 1
            else:
                Shell += 1
        
        return b_values

    def set_f(G, b_values):
        V, f = list(G.nodes), max(b_values.values())
        fi = {v: b_values[v] if all(b_values[v] >= b_values[vj] for vj in G.neighbors(v)) else 0 for v in V}

        while V:
            for vi in V:
                if fi[vi] == f:
                    for vj in G.neighbors(vi):
                        if fi[vi] - 1 > fi[vj]:
                            fi[vj] = fi[vi] - 1
            V.remove(vi)

        return fi

    b_values = set_b_values(G.copy())
    fi_values = set_f(G.copy(), b_values)

    S = {vi: sum(G.degree[vj] * (b_values[vj] + fi_values[vj]) for vj in G.neighbors(vi)) for vi in G.nodes()}
    HKS = {vi: sum(S[vj] for vj in G.neighbors(vi)) for vi in G.nodes()}
    return HKS

# Method 17 (SHKS)
def method17(G):
    # Precompute neighbor sets and node degrees for quick lookups.
    neighbors = {node: set(G.neighbors(node)) for node in G.nodes()}
    degrees = dict(G.degree())
    
    def compute_efficiency(G_local):
        n = G_local.number_of_nodes()
        if n <= 1:
            return 0
        total = 0.0
        # Precompute all-pairs shortest path lengths.
        for u, dist_dict in nx.all_pairs_shortest_path_length(G_local):
            for v, d in dist_dict.items():
                if d > 0:
                    total += 1 / d
        return total / (n * (n - 1))
    
    def compute_efficiency_centrality(G_local):
        efficiency_G = compute_efficiency(G_local)
        centrality = {}
        for node in G_local.nodes():
            # Remove node and compute efficiency for the resulting graph.
            G_copy = G_local.copy()
            G_copy.remove_node(node)
            e_copy = compute_efficiency(G_copy)
            centrality[node] = (efficiency_G - e_copy) / efficiency_G if efficiency_G > 0 else 0
        return centrality
    
    # Optimized pvu: if u is a neighbor of v, this value is 1/degree(v)
    def calculate_pvu(v, u):
        return 1 / degrees[v] if degrees[v] > 0 else 0

    # Compute the network-constrained coefficient for node v using precomputed neighbor sets.
    def network_constrained_coefficient(v):
        coeff = 0.0
        for u in neighbors[v]:
            # Outer term: always 1/deg(v) since u is a neighbor of v.
            outer_term = calculate_pvu(v, u)
            inner_sum = 0.0
            # Common neighbors of v and u.
            common = neighbors[v].intersection(neighbors[u])
            for w in common:
                term_vw = calculate_pvu(v, w)
                term_wu = calculate_pvu(w, u)  # u is in neighbors[w] since w is common.
                inner_sum += (term_vw * term_wu) ** 2
            coeff += outer_term + inner_sum
        return coeff

    # Compute efficiency centrality (if needed later; not used further in SHKS here)
    efficiency_centrality = compute_efficiency_centrality(G)
    
    # Compute the constraint coefficient for each node.
    Constraint_Coef = {node: network_constrained_coefficient(node) for node in G.nodes()}
    # Compute sh values with safe division.
    sh = {node: (1 / Constraint_Coef[node]) if Constraint_Coef[node] != 0 else 0 for node in G.nodes()}
    
    # Compute core numbers.
    ks = nx.core_number(G)
    alpha = 0.2
    I = {node: alpha * sh[node] + ks[node] for node in G.nodes()}
    # Compute C, IS, and finally SHKS using precomputed neighbor sets.
    C = {v: sum(I[v] + I[u] for u in neighbors[v]) for v in G.nodes()}
    IS = {v: sum(C[u] for u in neighbors[v]) for v in G.nodes()}
    SHKS = {v: sum(IS[u] for u in neighbors[v]) for v in G.nodes()}
    
    return SHKS


# method18: KSGC
def method18(G):
    # Precompute core numbers and degrees.
    ks = nx.core_number(G)
    ks_max = max(ks.values())
    ks_min = min(ks.values())
    k = dict(G.degree())
    
    n = G.number_of_nodes()
    nodes = list(G.nodes())
    node_index = {node: i for i, node in enumerate(nodes)}
    
    # Build arrays for core numbers and degrees.
    ks_array = np.array([ks[node] for node in nodes])
    k_array  = np.array([k[node] for node in nodes])
    
    # Compute the distance matrix using Floyd-Warshall.
    # This returns a NumPy array of shape (n, n) with distances;
    # unreachable pairs have np.inf.
    D_matrix = nx.floyd_warshall_numpy(G)
    
    # Compute the C matrix using vectorized operations.
    # Use a safe denominator to avoid division by zero.
    denom = (ks_max - ks_min) if (ks_max - ks_min) != 0 else 1
    C = np.exp((ks_array[:, None] - ks_array[None, :]) / denom)
    
    # Compute the F matrix:
    # F[i, j] = C[i, j] * (k[i] * k[j] / (d(i,j)^2)), if a path exists, else 0.
    with np.errstate(divide='ignore', invalid='ignore'):
        F = C * ((k_array[:, None] * k_array[None, :]) / (D_matrix ** 2))
        # For unreachable pairs (distance == inf), the division yields 0.
        F[np.isinf(D_matrix)] = 0
        # Ensure self-loops (i == j) are zero.
        np.fill_diagonal(F, 0)
    
    # Define the neighborhood radius based on the graph's diameter.
    diameter = nx.diameter(G)
    radius = int(0.5 * diameter)
    
    # Create a boolean mask for node pairs within the specified radius.
    # For each (i, j), mask[i, j] is True if the distance is less than or equal to radius.
    mask = D_matrix <= radius
    
    # Compute KSGC for each node by summing F over nodes in its neighborhood.
    # (F * mask) zeroes out contributions from nodes beyond the radius.
    KSGC_values = np.sum(F * mask, axis=1)
    
    # Build a result dictionary mapping node to its computed KSGC value.
    KSGC = {node: float(KSGC_values[node_index[node]]) for node in nodes}
    
    return KSGC


# Method 19 (Local Relative ASP)
def method19(G):
    def find_diameter(G):
        """
        Find the diameter (longest shortest path) of the graph G.
        """
        if nx.is_connected(G):
            return nx.diameter(G)
        else:
            lengths = dict(nx.all_pairs_shortest_path_length(G))
            diameter = 0
            for u in lengths:
                for v in lengths[u]:
                    if lengths[u][v] > diameter:
                        diameter = lengths[u][v]
            return diameter

    def calculate_asp(G):
        """
        Calculate the Average Shortest Path (ASP) for the graph G.
        """
        n = len(G.nodes)
        if n < 2:
            return 0
        
        if nx.is_connected(G):
            return nx.average_shortest_path_length(G)
        
        diameter = find_diameter(G)
        asp_sum = 0
        
        lengths = dict(nx.all_pairs_shortest_path_length(G))
        for u in G.nodes:
            for v in G.nodes:
                if u != v:
                    asp_sum += lengths[u].get(v, diameter)
        
        return asp_sum / (n * (n - 1))

    asp = calculate_asp(G)
    AC = {}

    for k in G.nodes():
        G_removed = G.copy()
        G_removed.remove_node(k)
        asp_removed = calculate_asp(G_removed)
        AC[k] = abs(asp_removed - asp) / asp if asp > 0 else 0

    return AC

def k_neighbors(G, node, radius):
        neighbors = set(nx.single_source_shortest_path_length(G, node, cutoff=radius).keys())
        neighbors.discard(node)  # Remove the node itself if present
        return neighbors
    
# Method 20 (InformationRank)
def method20(G):
    
    PROPA = {}
    Score = {}
    L = 2
    Miu = 0.2

    # Initialize PROPA and Score dictionaries
    for v in G.nodes():
        PROPA[v] = {}
        Score[v] = 0

    # Calculate PROPA and Score
    for v in G.nodes():
        neighbors = k_neighbors(G, v, L)
        for w in neighbors:
            PROPA[v][w] = 1
            path_length_counts = {}
            for path in nx.all_simple_paths(G, v, w, cutoff=L):
                length = len(path) - 1
                path_length_counts[length] = path_length_counts.get(length, 0) + 1

            for l in range(1, L + 1):
                pow_val = path_length_counts.get(l, 0)
                PROPA[v][w] *= (1 - Miu ** l) ** pow_val
            PROPA[v][w] = 1 - PROPA[v][w]
            Score[v] += PROPA[v][w]

    return Score

# method 21 (IS-PEW)
def method21(G):
    # Precompute expensive metrics
    triangles = nx.triangles(G)  # Dictionary {node: triangle_count}
    core_numbers = nx.core_number(G)  # Dictionary {node: core_number}
    degrees = dict(G.degree())  # Dictionary {node: degree}
    
    # Compute total triangles once
    TC = sum(triangles.values()) // 3 if sum(triangles.values()) > 0 else 1  # Avoid division by zero
    
    def compute_TP(v_A):
        return triangles[v_A] / TC
    
    def compute_InfE(v_A, v_B):
        common_neighbors = set(G[v_A]) & set(G[v_B])
        sum_DG_k = sum(degrees[k] for k in common_neighbors)
        return (degrees[v_A] * degrees[v_B]) / (1 + sum_DG_k)

    def compute_NIP(v_A):
        return sum(np.sqrt(degrees[v_a] * core_numbers[v_a]) for v_a in G[v_A])

    # Store edge weights in a dictionary to avoid recomputation
    edge_weights = {}

    def compute_EW(v_A, v_B):
        if (v_A, v_B) in edge_weights:
            return edge_weights[(v_A, v_B)]
        
        TP_v_A, TP_v_B = compute_TP(v_A), compute_TP(v_B)
        KS_v_A, KS_v_B = core_numbers[v_A], core_numbers[v_B]
        NIP_v_A, NIP_v_B = compute_NIP(v_A), compute_NIP(v_B)
        ED_v_A_v_B = 1 / compute_InfE(v_A, v_B)
        
        EW_value = ((KS_v_A * (1 + TP_v_A) * NIP_v_A) / ED_v_A_v_B) + ((KS_v_B * (1 + TP_v_B) * NIP_v_B) / ED_v_A_v_B)
        edge_weights[(v_A, v_B)] = EW_value  # Store result for future use
        edge_weights[(v_B, v_A)] = EW_value  # Store symmetric value
        return EW_value

    def compute_IS(v_A):
        return sum(compute_EW(v_A, v_B) for v_B in G[v_A])

    # Compute potential edge weights once
    for u, v in G.edges():
        G[u][v]['weight'] = compute_EW(u, v)

    # Compute IS values for all nodes
    IS_values = {v: compute_IS(v) for v in G.nodes()}
    
    return IS_values

# Method 22 (DKGM)
def method22(G):
    G_copy = G.copy()
    k_shell = {}
    k_star = {}
    k = 1
    iteration = 1

    while len(G_copy.nodes) > 0:
        iteration_nodes = []
        while True:
            nodes_to_remove = [node for node in G_copy.nodes if G_copy.degree[node] <= k]
            if not nodes_to_remove:
                break
            for node in nodes_to_remove:
                k_shell[node] = k
                iteration_nodes.append((node, iteration))
                G_copy.remove_node(node)
            iteration += 1

        if iteration_nodes:  # Ensure the list is not empty
            m = iteration_nodes[-1][1]  # Last iteration number
            for node, n in iteration_nodes:
                k_star[node] = k_shell[node] + (n / (m + 1))

        k += 1
        iteration = 1  # Reset iteration for next k-shell

    DK = {}
    DKGM = {}

    for node in k_shell:
        DK[node] = G.degree[node] + k_star.get(node, 0)  # Avoid KeyError

    R = 2  # Define the radius for DKGM computation
    for node in G.nodes():
        DKGM[node] = 0
        for neighbor in nx.single_source_shortest_path_length(G, node, cutoff=R):
            if node != neighbor:
                try:
                    d_ij = nx.shortest_path_length(G, source=node, target=neighbor)
                    DKGM[node] += DK[node] * DK.get(neighbor, 0) / d_ij**2  # Avoid KeyError
                except nx.NetworkXNoPath:
                    continue

    return DKGM


#method 23: modularity_vitality (MV)
def method23(G):
    """Detect communities using Leiden algorithm and compute modularity vitality for each node."""
    
    def detect_communities_leiden(G):
        """Detect communities using the Leiden algorithm.
        
        This function converts the NetworkX graph to an iGraph graph while preserving node names,
        runs the Leiden algorithm, and returns a dictionary mapping node names to community indices.
        """
        # Convert NetworkX graph to iGraph while preserving node names.
        ig_G = ig.Graph.TupleList(G.edges(), directed=False, vertex_name_attr='name')
        partition = la.find_partition(ig_G, la.ModularityVertexPartition)
        communities = {}
        # Map each iGraph vertex back to the original NetworkX node name.
        for idx, comm in enumerate(partition):
            for v in comm:
                communities[ig_G.vs[v]["name"]] = idx
        return communities

    def modularity(G, communities):
        """Compute the modularity of graph G with the given community partition."""
        M = G.number_of_edges()
        Q = 0
        for c in set(communities.values()):
            nodes_c = [n for n in G.nodes() if communities[n] == c]
            subgraph = G.subgraph(nodes_c)
            L_c = subgraph.number_of_edges()
            k_c = sum(G.degree(n) for n in nodes_c)
            Q += (L_c / M) - (k_c / (2 * M)) ** 2
        return Q

    def compute_hi_c(G, node, communities):
        """Compute h_i,c values for a node as per Equation (10)."""
        k_i = G.degree(node)
        hi_c = {}
        for c in set(communities.values()):
            # Count links from node to neighbors in community c.
            k_i_c = sum(1 for neighbor in G.neighbors(node) if communities[neighbor] == c)
            hi_c[c] = k_i_c + (k_i if communities[node] == c else 0)
        return hi_c

    def modularity_vitality(G, communities, node):
        """Compute the modularity vitality of a node using Equation (9)."""
        Q_initial = modularity(G, communities)
        if not G.has_node(node):
            return 0  

        M = G.number_of_edges()
        k_i = G.degree(node)
        c_i = communities[node]
        
        # Compute h_i,c for the given node.
        hi_c = compute_hi_c(G, node, communities)
        
        # Compute the total degree for each community.
        d_c = {c: sum(G.degree(n) for n in G.nodes() if communities[n] == c)
               for c in set(communities.values())}

        # Get nodes in the same community as node.
        community_nodes = [n for n in G.nodes() if communities[n] == c_i]
        M_internal = G.subgraph(community_nodes).number_of_edges()
        k_i_internal = sum(1 for neighbor in G.neighbors(node) if communities[neighbor] == c_i)
        
        # Compute the sum term over all communities.
        sum_term = sum((d_c[c] - hi_c[c]) ** 2 for c in hi_c)
        
        # Compute updated modularity based on Equation (9).
        Q_updated = (M_internal - k_i_internal) / (M - k_i) - (1 / (4 * (M - k_i) ** 2)) * sum_term

        return Q_initial - Q_updated

    # Detect communities using the Leiden algorithm.
    communities = detect_communities_leiden(G)

    # Compute modularity vitality for each node.
    mod_vit = {node: modularity_vitality(G, communities, node) for node in G.nodes()}
    
    return mod_vit

#method 24: Global-and-Local Centrality (GLC)
def method24(G, lam=1.0):
    """
    Compute the Global-and-Local Centrality (GLC) of each node in graph G.
    
    The method first clusters the network using a potential-based approach,
    then selects global critical nodes from each cluster, computes local influence
    based on k-shell values, and finally calculates the overall GLC value.
    
    Parameters:
        G   : networkx.Graph
              The input graph.
        lam : float (default=1.0)
              The fraction (lambda) of nodes that must be covered by clusters.
    
    Returns:
        GLC : dict
              A dictionary mapping each node in G to its GLC centrality value.
    """
    # --- Step 1: Clustering and Global Critical Node Selection ---
    clusters = []       # List to hold all clusters
    assigned = set()    # Set of nodes that have been assigned or marked
    total_nodes = G.number_of_nodes()
    
    # Helper function: Compute the potential of a node as defined in Eq.(10)
    def compute_potential(node):
        # Potential of node = degree(node) * (sum over j in N(node) of kin(j))
        # where kin(j) = number of neighbors of j that are in {node} U N(node)
        nbrs = set(G.neighbors(node))
        target_set = nbrs.union({node})
        s = 0
        for j in nbrs:
            s += sum(1 for nbr in G.neighbors(j) if nbr in target_set)
        return G.degree(node) * s

    # Continue clustering until the number of assigned nodes reaches lam * total_nodes
    while len(assigned) < total_nodes * lam:
        # Compute potentials for all unassigned nodes.
        potentials = {}
        for node in G.nodes():
            if node not in assigned:
                potentials[node] = compute_potential(node)
        if not potentials:
            break  # All nodes have been assigned
        
        # Select seed node with maximum potential.
        seed = max(potentials, key=potentials.get)
        pcmax = potentials[seed]
        new_cluster = set([seed])
        assigned.add(seed)
        
        # Include neighbors of the seed with potential >= pcmax/2.
        for neighbor in G.neighbors(seed):
            if neighbor not in assigned and compute_potential(neighbor) >= pcmax / 2:
                new_cluster.add(neighbor)
                assigned.add(neighbor)
                
        # Expand the cluster iteratively (three degrees of influence).
        for _ in range(3):
            # Gather neighbors of the current cluster (excluding nodes already in the cluster)
            cluster_neighbors = set()
            for node in new_cluster:
                for nbr in G.neighbors(node):
                    if nbr not in new_cluster:
                        cluster_neighbors.add(nbr)
            # Process neighbors in order of increasing degree.
            cluster_neighbors = sorted(cluster_neighbors, key=lambda x: G.degree(x))
            
            added_flag = False
            for candidate in cluster_neighbors:
                # Count the number of edges from candidate to nodes in new_cluster (kin)
                kin = sum(1 for nbr in G.neighbors(candidate) if nbr in new_cluster)
                # Count the remaining edges from candidate to nodes outside new_cluster (kout)
                kout = sum(1 for nbr in G.neighbors(candidate) if nbr not in new_cluster)
                if kin >= kout:
                    new_cluster.add(candidate)
                    assigned.add(candidate)
                    added_flag = True
            if not added_flag:
                break  # No more nodes can be added in this iteration
        
        # Mark all neighbors of the new cluster as assigned to avoid re-selection.
        for node in new_cluster:
            for nbr in G.neighbors(node):
                assigned.add(nbr)
        clusters.append(new_cluster)
    
    # From each cluster, select the global critical node (node with highest degree).
    global_critical = []
    for cluster in clusters:
        if cluster:
            gc = max(cluster, key=lambda node: G.degree(node))
            global_critical.append(gc)
    
    # --- Step 2: Local Influence Calculation ---
    # Compute the k-shell (core) numbers for all nodes.
    core_numbers = nx.core_number(G)
    
    # Compute local influence LI for each node:
    # LI(node) = sum_{j in N(node)} k_shell(j)
    LI = {}
    for node in G.nodes():
        li = 0
        for nbr in G.neighbors(node):
            li += core_numbers[nbr]
        LI[node] = li

    # --- Step 3: Overall GLC Centrality Calculation ---
    # For each node v, compute:
    #   GlobalFactor(v) = sum_{u in global_critical} (LI(u) / (2 * max(d(v,u),1)))
    #   GLC(v) = LI(v) * GlobalFactor(v)
    GLC = {}
    for v in G.nodes():
        global_factor = 0
        for u in global_critical:
            try:
                d = nx.shortest_path_length(G, source=v, target=u)
            except nx.NetworkXNoPath:
                d = float('inf')
            d = max(d, 1)  # Avoid division by zero
            global_factor += LI[u] / (2 * d)
        GLC[v] = LI[v] * global_factor

    return GLC

# Method 25 (Weighted K-shell)
def method25(G):
    c1 = 0.1
    c2 = 0.4
    deg = dict(G.degree())
    ks = nx.core_number(G)
    ksdw = dict()

    for vi in G.nodes():
        ksdw[vi] = 0
        for vj in G.neighbors(vi):
            ksdw[vi] += (c1 * deg[vi] + c2 * ks[vi]) * (c1 * deg[vj] + c2 * ks[vj])

    return ksdw

# method 26:EFFD centrality
def method26(G):
    def calculate_effective_distance(G):
        degree = dict(G.degree())
        
        # Step 1: Calculate the probability P n|m
        probability = {(m, n): (1 / degree[m]) if G.has_edge(m, n) else 0 
                       for m in G.nodes for n in G.nodes if m != n}

        # Step 2: Calculate the effective distance D_{n|m} for directly connected nodes
        effective_distance = {(m, n): 1 - np.log2(p) if p > 0 else float('inf') 
                              for (m, n), p in probability.items()}

        # Step 3: Calculate the effective distance for indirectly connected nodes using the shortest path
        all_pairs_shortest_path_length = dict(nx.all_pairs_dijkstra_path_length(G))
        for m in G.nodes:
            for n in G.nodes:
                if m != n:
                    shortest_path_length = all_pairs_shortest_path_length[m].get(n, float('inf'))
                    if shortest_path_length != float('inf'):
                        effective_distance[(m, n)] = shortest_path_length

        return effective_distance

    def calculate_interaction_scores(G, effective_distance):
        interaction_scores = {}
        for (i, j), d in effective_distance.items():
            k_i = G.degree[i]
            k_j = G.degree[j]
            interaction_scores[(i, j)] = (k_i * k_j) / (d ** 2)
        return interaction_scores

    def compute_effg_centrality(G, interaction_scores):
        effg_centrality = {node: 0 for node in G.nodes()}
        for (i, j), score in interaction_scores.items():
            effg_centrality[i] += score
            effg_centrality[j] += score
        return effg_centrality

    # Calculate Effective Distance
    effective_distance = calculate_effective_distance(G)

    # Calculate Interaction Scores
    interaction_scores = calculate_interaction_scores(G, effective_distance)

    # Calculate EffG Centrality
    effg_centrality = compute_effg_centrality(G, interaction_scores)
    
    return effg_centrality






#methods = {
#    "Degree Centrality": method1,
#    "Betweenness Centrality": method2,
#    "Closeness Centrality": method3,
#    "K-shell": method4,
#    "Cnc Plus": method5,
#    "Local H-index": method6,
#    "DNC": method7,
#    "ECLGC": method8,
#    "Centripetal Centrality": method9,
#    "EDBC" : method10,
#    "LGC" : method11,
#    "SEGM" : method12,
#    "MCGM" : method13,
#    "HVGC" : method14,
#    "BaseGM" : method15,
#    "HKS" : method16,
#    "SHKS" : method17,
#    "KSGC" : method18,
#    "Local Relative ASP" : method19,
#    "Information Rank" : method20,
#    "IS-PEW" : method21,
#    "DKGM" : method22,
#    "modularity vitality":method23,
#    "Global-and-Local Centrality (GLC)":method24,
#    "Weighted K-shell" : method25,
#    "EFFD gravity" : method26,
# }

# List of methods to be applied
methods = [globals()[f'method{i}'] for i in range(1, 27)]

dataset_folder = 'dataset'
beta_values = [0.02, 0.05, 0.08, 0.10, 0.13, 0.15, 0.18, 0.20, 0.22, 0.25, 0.30]
k_factors = [0.01, 0.03, 0.05, 0.07, 0.085, 0.10]
f_values = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1]

for dataset_filename in os.listdir(dataset_folder):
    if dataset_filename.endswith('.xlsx'):
        dataset_path = os.path.join(dataset_folder, dataset_filename)
        dataset_name = os.path.splitext(dataset_filename)[0]
        print(f"\nProcessing dataset: {dataset_name}...\n")
        
        df_edges = pd.read_excel(dataset_path, sheet_name='Sheet1')
        G = nx.from_pandas_edgelist(df_edges, source='source_column', target='target_column')
        G.remove_edges_from(nx.selfloop_edges(G))
        
        df_sir = pd.read_excel(dataset_path, sheet_name='SIR')
        df_sir.set_index('Node', inplace=True)
        
        num_nodes = len(G.nodes())
        results = {}
        method_results = {}

        for i, method in enumerate(methods, start=1):
            method_name = f'method{i}'
            print(f"Applying {method_name} on {dataset_name}...")
            try:
                ranked_nodes, exec_time = measure_execution_time(method, G)
                monotonicity = compute_monotonicity(G, ranked_nodes)
                method_results[method_name] = {
                    'ranked_nodes': ranked_nodes,
                    'execution_time': exec_time,
                    'monotonicity': monotonicity
                }
            except Exception as e:
                print(f"Error in {method_name} for {dataset_name}: {e}")
                method_results[method_name] = {
                    'ranked_nodes': [],
                    'execution_time': None,
                    'monotonicity': None,
                    'error': str(e)
                }

        for beta in beta_values:
            beta_column = f'Beta_{beta:.2f}'
            if beta_column not in df_sir.columns:
                print(f"Warning: {beta_column} not found in {dataset_name}. Skipping beta {beta}...")
                continue
            
            spread_power = df_sir[beta_column].to_dict()
            sigma = sorted(spread_power.keys(), key=lambda x: spread_power[x], reverse=True)
            
            for method_name, method_data in method_results.items():
                if 'error' in method_data:
                    results[(method_name, beta)] = {'Beta': beta, 'Kendall Tau': None, 'P-Value': None}
                    continue
                
                ranked_nodes = method_data['ranked_nodes']
                tau, p_value = kendalltau(
                    [spread_power[node] for node in sigma],
                    [spread_power[node] for node, _ in ranked_nodes]
                )
                
                R = [node for node, _ in ranked_nodes]
                jaccard_scores = {}
                si_scores = {}
                rbo_scores = {}
                mrr_scores = compute_mrr(sigma, R, k_factors)
                
                for k_factor in k_factors:
                    k = max(1, int(k_factor * num_nodes))
                    top_k_sigma = set(sigma[:k])
                    top_k_R = set(R[:k])
                    jaccard_scores[f'Jaccard k={k}'] = compute_jaccard_similarity(top_k_sigma, top_k_R)
                
                for f in f_values:
                    f_count = max(1, int(f * num_nodes))
                    top_f_nodes = [node for node, _ in ranked_nodes][:f_count]
                    si_scores[f'SI {f:.2f}'] = sum(spread_power[node] for node in top_f_nodes) / (f * num_nodes)
                    rbo_scores[f'RBO {f:.2f}'] = compute_rbo(sigma[:f_count], R[:f_count])
                
                results[(method_name, beta)] = {
                    'Beta': beta,
                    'Kendall Tau': tau,
                    'P-Value': p_value,
                    **jaccard_scores,
                    **si_scores,
                    **rbo_scores,
                    **mrr_scores
                }
        
        data = []
        for (method_name, beta), result in results.items():
            row = [
                dataset_name,  # Include dataset name in output
                method_name,
                method_results[method_name]['execution_time'],
                method_results[method_name].get('monotonicity', 'N/A'),
                result.get('Beta'),
                result.get('Kendall Tau'),
                result.get('P-Value')
            ] + [result.get(f'Jaccard k={int(k_factor * num_nodes)}') for k_factor in k_factors] + \
              [result.get(f'SI {f:.2f}') for f in f_values] + \
              [result.get(f'RBO {f:.2f}') for f in f_values] + \
              [result.get(f'MRR {k_factor:.2f}') for k_factor in k_factors]
            
            data.append(row)
        
        expected_columns = ['Dataset', 'Method', 'Execution Time', 'Monotonicity', 'Beta', 'Kendall Tau', 'P-Value'] + \
                           [f'Jaccard k={int(k_factor * num_nodes)}' for k_factor in k_factors] + \
                           [f'SI {f:.2f}' for f in f_values] + \
                           [f'RBO {f:.2f}' for f in f_values] + \
                           [f'MRR {k_factor:.2f}' for k_factor in k_factors]
        
        result_df = pd.DataFrame(data, columns=expected_columns)
        
        output_file = f'{dataset_name}_results.xlsx'
        with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
            df_edges.to_excel(writer, sheet_name='Sheet1', index=False)
            df_sir.to_excel(writer, sheet_name='SIR')
            result_df.to_excel(writer, sheet_name='Results', index=False)
        
        print(f"✅ Completed processing: {dataset_name}. Results saved in {output_file}\n")


Processing dataset: EgoFacebook(SIR)...

Applying method1 on EgoFacebook(SIR)...
Applying method2 on EgoFacebook(SIR)...
Applying method3 on EgoFacebook(SIR)...
Applying method4 on EgoFacebook(SIR)...
Applying method5 on EgoFacebook(SIR)...
Applying method6 on EgoFacebook(SIR)...
Applying method7 on EgoFacebook(SIR)...
Applying method8 on EgoFacebook(SIR)...
Applying method9 on EgoFacebook(SIR)...
Applying method10 on EgoFacebook(SIR)...
Applying method11 on EgoFacebook(SIR)...
Applying method12 on EgoFacebook(SIR)...
Applying method13 on EgoFacebook(SIR)...
Applying method14 on EgoFacebook(SIR)...
Applying method15 on EgoFacebook(SIR)...
Applying method16 on EgoFacebook(SIR)...
Applying method17 on EgoFacebook(SIR)...


# Optimized Version

In [8]:
import math
import os
import pandas as pd
import networkx as nx
import numpy as np
import time
import openpyxl
from openpyxl import Workbook
from collections import deque
import random
from scipy.stats import kendalltau
import igraph as ig
import leidenalg as la

def measure_execution_time(method, G):
    start_time = time.time()
    result = method(G)
    ranked_nodes = sorted(result.items(), key=lambda x: x[1], reverse=True)
    end_time = time.time()
    execution_time = end_time - start_time
    return ranked_nodes, execution_time

def compute_monotonicity(G, ranked_nodes):
    ranks = [score for node, score in ranked_nodes]
    unique_ranks = list(set(ranks))
    n = G.number_of_nodes()
    nr_dict = {rank: ranks.count(rank) for rank in unique_ranks}
    nr_sum = sum(nr * (nr - 1) for nr in nr_dict.values())
    M = (1 - nr_sum / (n * (n - 1))) ** 2
    return M

def compute_mrr(ground_truth, ranked_list, top_ratios):
    mrr_scores = {}
    num_nodes = len(ground_truth)
    
    for ratio in top_ratios:
        k = max(1, int(ratio * num_nodes))
        reciprocal_ranks = []
        
        for node in ground_truth[:k]:
            if node in ranked_list:
                rank = ranked_list.index(node) + 1
                reciprocal_ranks.append(1 / rank)
        
        mrr_scores[f'MRR {ratio:.2f}'] = sum(reciprocal_ranks) / k if reciprocal_ranks else 0
    
    return mrr_scores


def compute_rbo(sigma, R, alpha=0.9):
    """Compute the Rank-Biased Overlap (RBO) between two ranking lists."""
    def A(sigma, R, f):
        sigma_f = set(sigma[:f])
        R_f = set(R[:f])
        if len(sigma_f) == 0 and len(R_f) == 0:
            return 1
        return len(sigma_f & R_f) / len(sigma_f | R_f)

    n = max(len(sigma), len(R))
    rbo = (1 - alpha) * sum(alpha**(f - 1) * A(sigma, R, f) for f in range(1, n + 1))
    return rbo

def compute_jaccard_similarity(set1, set2):
    return len(set1 & set2) / len(set1 | set2) if set1 | set2 else 0

# Define methods
# method1: Degree
def method1(G):
    return nx.degree_centrality(G)

# method2: Betweenness Centrality
def method2(G, epsilon=0.1, delta=0.1, c=1.0):
    """
    Computes an approximate betweenness centrality for all nodes in G.
    
    Parameters:
    G : networkx.Graph
        The input graph.
    epsilon : float, optional
        Error tolerance parameter.
    delta : float, optional
        Probability of failure parameter.
    c : float, optional
        Scaling constant.
    
    Returns:
    dict
        Dictionary where keys are nodes and values are their approximate betweenness centrality.
    """
    # Step 1: Initialize betweenness values
    betweenness = {v: 0 for v in G.nodes()}
    
    # Step 2: Compute estimated number of samples r
    diameter = nx.diameter(G) if nx.is_connected(G) else max(nx.eccentricity(G).values())
    r = int((c / (epsilon ** 2)) * (math.log2(max(diameter - 2, 1)) + math.log(1 / delta)))
    
    nodes = list(G.nodes())
    
    # Step 3: Perform sampling
    for _ in range(r):
        u, v = random.sample(nodes, 2)
        shortest_paths = list(nx.all_shortest_paths(G, source=u, target=v))
        
        if len(shortest_paths) > 1:
            path = random.choice(shortest_paths)  # Randomly select one shortest path
            
            for z in path[1:-1]:  # Exclude source (u) and target (v)
                betweenness[z] += 1 / r  # Update betweenness estimation
    
    return betweenness


# method3: Closeness Centrality
def method3(G):
    return nx.closeness_centrality(G)

# method4: k-shell
def method4(G):
    return nx.core_number(G)

def CnC(G):
    ks = nx.core_number(G)
    cnc = {vi: sum(ks[vj] for vj in G.neighbors(vi)) for vi in G.nodes()}
    return cnc

# method5: CncPlus
def method5(G):
    ks = nx.core_number(G)
    cnc = CnC(G)
    cnc_plus = {vi: sum(cnc[vj] for vj in G.neighbors(vi)) for vi in G.nodes()}
    return cnc_plus

def HIndex(G):
    h_index = {}
    for node in G.nodes():
        neighbors = list(G.neighbors(node))
        degrees = sorted([G.degree(neighbor) for neighbor in neighbors], reverse=True)
        h = 0
        for i, degree in enumerate(degrees):
            if degree >= i + 1:
                h = i + 1
            else:
                break
        h_index[node] = h
    return h_index

# method6: Local H-Index
def method6(G):
    HI = HIndex(G)
    LHI = {vi: HI[vi] + sum(HI[vj] for vj in G.neighbors(vi)) for vi in G.nodes()}
    return LHI

# method7: DNC
def method7(G):
    SLC = {}
    k = dict(nx.degree(G))
    C = nx.clustering(G)
    for i in G.nodes():
        SLC[i] = sum(C[j] for j in G.neighbors(i))
    
    DNC = {}
    alpha = 1
    for i in G.nodes():
        DNC[i] = k[i] + alpha * SLC[i]
    return DNC

# Method 8: ECLGC
def method8(G):
    # Precompute clustering coefficients
    CLC = nx.clustering(G)
    
    # Precompute neighbors for each node
    neighbors = {node: list(G.neighbors(node)) for node in G.nodes()}
    
    # Precompute shortest path lengths for all node pairs
    sp_lengths = dict(nx.all_pairs_shortest_path_length(G))
    
    # Compute CLGC (not used later, but computed as in the original code)
    CLGC = {}
    for i in G.nodes():
        s = 0.0
        for j in G.nodes():
            if i == j:
                continue
            if j in sp_lengths[i]:
                s += math.sqrt(CLC[j]) / sp_lengths[i][j]
        CLGC[i] = s * CLC[i]
    
    # Compute ECLGC
    ECLGC = {}
    for v in G.nodes():
        sum1 = 0.0
        sum3 = 0.0
        neighbors_v = neighbors[v]
        if not neighbors_v:  # if no neighbors, ECLGC is 0
            ECLGC[v] = 0
            continue
        
        for u in G.nodes():
            if u == v:
                continue
            if u in sp_lengths[v]:
                # Precompute sum2 for u's neighbors
                neighbors_u = neighbors[u]
                lu = len(neighbors_u) if neighbors_u else 1  # avoid division by zero
                sum2 = 0.0
                for j in neighbors_u:
                    sum2 += CLC[j] / lu
                
                sum1 += CLC[u] / len(neighbors_v)
                d = sp_lengths[v][u]  # shortest path length from v to u
                sum3 += math.sqrt(CLC[u] + sum2) / d
        ECLGC[v] = sum1 * sum3
    return ECLGC



def get_neighborhood(G, node, radius):
    neighborhood = nx.single_source_shortest_path_length(G, node, cutoff=radius)
    neighborhood.pop(node, None)
    return list(neighborhood.keys())


# method9: CenC
def compute_alpha(graph):
    size_of_V = graph.number_of_nodes()
    # Ensure order_of_magnitude is at least 1 to avoid division by zero.
    order_of_magnitude = max(int(math.log10(size_of_V)), 1)
    alpha = size_of_V / order_of_magnitude
    return alpha

def method9(G, m=1):
    alpha = compute_alpha(G)
    # Cap alpha to avoid math range error in math.exp (overflow for very large alpha)
    capped_alpha = min(alpha, 700)
    
    degree_centrality = nx.degree_centrality(G)
    clustering_coeff = nx.clustering(G)
    kshell = nx.core_number(G)
    
    centrality = {}
    
    for i in G.nodes():
        cenC_i = 0
        
        # Precompute the m-neighborhood and distances from i
        sp_lengths_i = nx.single_source_shortest_path_length(G, i, cutoff=m)
        m_neighborhood = set(sp_lengths_i.keys())
        m_neighborhood.discard(i)  # Exclude the node itself
        
        for j in m_neighborhood:
            d_ij = sp_lengths_i[j]  # Use precomputed distance
            if d_ij == 0:
                continue
            
            DC_i = degree_centrality[i]
            C_i = clustering_coeff[i]
            k_s_i = kshell[i]
            k_s_j = kshell[j]
            
            # Compute term1 with a safe exponentiation
            term1 = (math.exp(capped_alpha) * DC_i + (1 / C_i if C_i != 0 else 0))
            term2 = k_s_j / (abs(k_s_i - k_s_j) + 1)
            
            cenC_i += (4 * math.pi**2) * (term1 / (d_ij**2)) * term2
        
        centrality[i] = cenC_i
    
    return centrality

def k_shell(graph):
    return nx.core_number(graph)

def evaluate_power_k_shell(k_si, k_sj):
    return math.sqrt(k_si + k_sj)

def compute_P_ji(graph, node_i, node_j):
    try:
        a_ji = graph[node_j][node_i]['weight'] if 'weight' in graph[node_j][node_i] else 1
        total_a_ji = sum(graph[node_j][nbr]['weight'] if 'weight' in graph[node_j][nbr] else 1 for nbr in graph.neighbors(node_j))
        return a_ji / total_a_ji
    except KeyError:
        return 0

def evaluate_ED(graph, node_i, node_j):
    P_ji = compute_P_ji(graph, node_i, node_j)
    if P_ji == 0:
        return 1  # Prevent log(0)
    return max(1 - math.log(P_ji), 1)

def evaluate_influence(graph, node_i, node_j, k_shell_values):
    alpha_ij = evaluate_power_k_shell(k_shell_values[node_i], k_shell_values[node_j])
    ED_ij = evaluate_ED(graph, node_i, node_j)
    degree_vi = graph.degree(node_i)
    return alpha_ij * degree_vi / ED_ij

# EDBC
def method10(G):
    k_shell_values = k_shell(G)
    EDBC = {u: 0 for u in G.nodes()}
    
    for u in G.nodes():
        for v in G.neighbors(u):
            for neighbor_u in G.neighbors(u):
                EDBC[v] += evaluate_influence(G, v, neighbor_u, k_shell_values)
    
    return EDBC

# Method 11 (LGC)
def method11(G):
    # Precompute degrees and neighbor lists.
    K = dict(G.degree())
    neighbor_dict = {node: list(G.neighbors(node)) for node in G.nodes()}
    
    # Precompute LC values for each node.
    LC = {}
    for i in G.nodes():
        LC[i] = sum(K[j] for j in neighbor_dict[i]) * 2 + K[i] ** 2 + K[i]
    
    LGC = {}
    radius = 3
    # For each node, compute the distances to nodes within the given radius.
    for vi in G.nodes():
        # Use a single BFS to get distances up to radius.
        sp_lengths = nx.single_source_shortest_path_length(G, vi, cutoff=radius)
        total = 0.0
        for vj, d in sp_lengths.items():
            if vi == vj or d == 0:
                continue  # Skip self-loops or zero distance.
            total += LC[vi] * LC[vj] / (d ** 2)
        LGC[vi] = total
    
    return LGC

# Method 12 (SEGM)
def method12(G):
    k = dict(G.degree())
    I = dict()
    E = dict()
    SE = dict()

    for node in G.nodes:
        I[node] = 0

    for i in G.nodes:
        sum_kj = 0
        for j in G.neighbors(i):
            sum_kj += k[j]
        if sum_kj != 0:
            I[i] = k[i] / sum_kj
        sum_e = 0
        for j in G.neighbors(i):
            if I[j] != 0:
                sum_e += I[j] * np.log(I[j])
        E[i] = -1 * sum_e
        SE[i] = np.exp(E[i]) * k[i]

    SEGM = dict()
    R = 3
    for i in G.nodes:
        SEGM[i] = 0
        neighbors = get_neighborhood(G, i, R)
        for j in neighbors:
            if nx.shortest_path_length(G, i, j) != 0:
                SEGM[i] += (SE[i] * SE[j]) / (nx.shortest_path_length(G, i, j) ** 2)

    return SEGM

# Method 13 (MCGM)
def method13(G):
    # Compute degree and core number
    k = dict(G.degree())
    ks = nx.core_number(G)
    
    # Compute eigenvector centrality with increased iterations.
    try:
        x = nx.eigenvector_centrality(G, max_iter=500, tol=1e-06)
    except nx.PowerIterationFailedConvergence:
        # Fallback to NumPy-based eigenvector centrality if convergence fails.
        x = nx.eigenvector_centrality_numpy(G)
    
    # Convert centrality measures to arrays for median and max computations.
    k_values = np.array(list(k.values()))
    ks_values = np.array(list(ks.values()))
    x_values = np.array(list(x.values()))
    
    # Compute medians and maximum values.
    k_mid = np.median(k_values)
    ks_mid = np.median(ks_values)
    x_mid = np.median(x_values)
    
    k_max = np.max(k_values)
    ks_max = np.max(ks_values)
    x_max = np.max(x_values)
    
    # Calculate alpha, ensuring no division by zero.
    if ks_mid == 0:
        alpha = 0
    else:
        alpha = max(k_mid / k_max, x_mid / x_max) / (ks_mid / ks_max)
    
    MCGM = {}
    R = 3  # Radius for considering node pairs.
    
    # Precompute shortest path lengths for all nodes with a cutoff of R.
    sp_lengths = dict(nx.all_pairs_shortest_path_length(G, cutoff=R))
    
    for i in G.nodes():
        MCGM[i] = 0
        # Iterate over nodes reachable from i within cutoff R.
        for j, d in sp_lengths.get(i, {}).items():
            if i == j or d == 0:
                continue
            # Only consider nodes within radius R.
            if d <= R:
                # Compute contribution from node j.
                term_i = (k[i] / k_max + alpha * ks[i] / ks_max + x[i] / x_max)
                term_j = (k[j] / k_max + alpha * ks[j] / ks_max + x[j] / x_max)
                MCGM[i] += term_i * term_j / (d ** 2)
    
    return MCGM

# Method 14 (HVGC)
def compute_H_index(G, i, degree_dict, neighbors):
    # Use precomputed degrees for neighbors.
    neighbor_degrees = [degree_dict[nb] for nb in neighbors[i]]
    neighbor_degrees.sort(reverse=True)
    H_index = 0
    for idx, deg in enumerate(neighbor_degrees):
        if deg >= idx + 1:
            H_index = idx + 1
        else:
            break
    return H_index

def compute_H_v(G, i, degree_dict, neighbors):
    H_i = compute_H_index(G, i, degree_dict, neighbors)
    # Sum degrees of neighbors meeting the H_index threshold.
    H_v_i = sum(degree_dict[j] for j in neighbors[i] if degree_dict[j] >= H_i)
    return H_v_i

def compute_c_i(i, neighbors, sum_weights, edge_weights):
    # Compute c_i based on neighbors of node i.
    c_i = 0
    # Use the precomputed neighbor list for i.
    neighbors_i = neighbors[i]
    for j in neighbors_i:
        # p_ij = weight(i,j) / (sum of weights from i)
        p_ij = edge_weights.get((i, j), 1) / sum_weights[i] if sum_weights[i] != 0 else 0
        # common neighbors between i and j
        common_neighbors = set(neighbors_i).intersection(neighbors[j])
        sum_piw_pwj = 0
        for w in common_neighbors:
            p_iw = edge_weights.get((i, w), 1) / sum_weights[i] if sum_weights[i] != 0 else 0
            p_wj = edge_weights.get((w, j), 1) / sum_weights[w] if sum_weights[w] != 0 else 0
            sum_piw_pwj += p_iw * p_wj
        term = p_ij + sum_piw_pwj
        c_i += term ** 2
    return c_i

def compute_HVGC(G, i, H_v, R, neighbors, sum_weights, edge_weights):
    # Compute c_i using the optimized helper.
    c_i = compute_c_i(i, neighbors, sum_weights, edge_weights)
    HVGC_i = 0
    # Use BFS (single_source_shortest_path_length) with cutoff R to consider only nearby nodes.
    sp_lengths = nx.single_source_shortest_path_length(G, i, cutoff=R)
    for j, d_ij in sp_lengths.items():
        if j == i:
            continue
        # d_ij is guaranteed > 0; add contribution from node j.
        HVGC_i += (math.exp(-c_i) * H_v[i] * H_v[j]) / (d_ij ** 2)
    return HVGC_i

def method14(G):
    """
    Optimized HVGC computation.
    
    Precomputes node degrees, neighbors, edge weights, and sum of weights.
    Then computes H_v for each node and finally HVGC using a BFS-based approach.
    """
    # Precompute node degrees and neighbors
    degree_dict = dict(G.degree())
    neighbors = {i: list(G.neighbors(i)) for i in G.nodes()}
    
    # Precompute the sum of weights for each node (using weight attribute or defaulting to 1)
    sum_weights = {}
    for i in G.nodes():
        sum_weights[i] = sum(G[i][w].get('weight', 1) for w in neighbors[i]) if neighbors[i] else 0
    
    # Precompute edge weights (store for both directions in the undirected graph)
    edge_weights = {}
    for i, j in G.edges():
        w = G[i][j].get('weight', 1)
        edge_weights[(i, j)] = w
        edge_weights[(j, i)] = w

    # Compute H_v for each node
    H_v = {}
    for i in G.nodes():
        H_v[i] = compute_H_v(G, i, degree_dict, neighbors)

    R = 2  # Neighborhood radius
    HVGC = {}
    for i in G.nodes():
        HVGC[i] = compute_HVGC(G, i, H_v, R, neighbors, sum_weights, edge_weights)
    
    return HVGC



# Method15 (BaseGM)
def method15(G):
    def get_neighborhood(G, node, radius):
        neighborhood = nx.single_source_shortest_path_length(G, node, cutoff=radius)
        neighborhood.pop(node, None)
        return list(neighborhood.keys())

    gravity = dict()
    kshell = nx.core_number(G)
    for vi in G.nodes():
        gravity[vi] = 0
        neighbors = get_neighborhood(G, vi, 3)
        for vj in neighbors:
            gravity[vi] += ((kshell[vi] * kshell[vj]) / nx.shortest_path_length(G, vi, vj) ** 2)

    gravity_plus = dict()
    for vi in G.nodes():
        gravity_plus[vi] = 0
        for vj in G.neighbors(vi):
            gravity_plus[vi] += gravity[vj]

    return gravity_plus

# Method 16 (HKS)
def method16(G):
    def set_b_values(G):
        b, Shell, b_values = 1, 1, {}
        while G.number_of_nodes() > 0:
            flag = False
            for v in list(G.nodes):
                if G.degree[v] <= Shell:
                    b_values[v] = b
                    flag = True
            
            if flag:
                G.remove_nodes_from([v for v in list(G.nodes) if b_values.get(v) == b])
                b += 1
            else:
                Shell += 1
        
        return b_values

    def set_f(G, b_values):
        V, f = list(G.nodes), max(b_values.values())
        fi = {v: b_values[v] if all(b_values[v] >= b_values[vj] for vj in G.neighbors(v)) else 0 for v in V}

        while V:
            for vi in V:
                if fi[vi] == f:
                    for vj in G.neighbors(vi):
                        if fi[vi] - 1 > fi[vj]:
                            fi[vj] = fi[vi] - 1
            V.remove(vi)

        return fi

    b_values = set_b_values(G.copy())
    fi_values = set_f(G.copy(), b_values)

    S = {vi: sum(G.degree[vj] * (b_values[vj] + fi_values[vj]) for vj in G.neighbors(vi)) for vi in G.nodes()}
    HKS = {vi: sum(S[vj] for vj in G.neighbors(vi)) for vi in G.nodes()}
    return HKS

# Method 17 (SHKS)
def method17(G):
    # Precompute neighbor sets and node degrees.
    neighbors = {node: set(G.neighbors(node)) for node in G.nodes()}
    degrees = dict(G.degree())
    
    # Precompute pvu values for all nodes: 1/degree if degree > 0 else 0.
    pvu_value = {node: 1/deg if deg > 0 else 0 for node, deg in degrees.items()}
    
    # Compute the network-constrained coefficient for node v.
    def network_constrained_coefficient(v):
        coeff = 0.0
        pvu_v = pvu_value[v]
        for u in neighbors[v]:
            # Outer term is simply 1/deg(v)
            outer_term = pvu_v
            # For common neighbors between v and u, sum (1/deg(v)^2 * (1/deg(w))^2)
            common = neighbors[v].intersection(neighbors[u])
            inner_sum = (pvu_v ** 2) * sum(pvu_value[w] ** 2 for w in common)
            coeff += outer_term + inner_sum
        return coeff

    # Compute constraint coefficient for each node.
    Constraint_Coef = {node: network_constrained_coefficient(node) for node in G.nodes()}
    # Compute sh values safely.
    sh = {node: 1 / Constraint_Coef[node] if Constraint_Coef[node] != 0 else 0 for node in G.nodes()}
    
    # Compute core numbers.
    ks = nx.core_number(G)
    alpha = 0.2
    # Compute influence I for each node.
    I = {node: alpha * sh[node] + ks[node] for node in G.nodes()}
    # Compute C[v] = I[v] plus the sum of I over its neighbors.
    C = {v: I[v] + sum(I[u] for u in neighbors[v]) for v in G.nodes()}
    # Compute IS[v] as the sum of C over neighbors of v.
    IS = {v: sum(C[u] for u in neighbors[v]) for v in G.nodes()}
    # Finally, compute SHKS[v] as the sum of IS over neighbors of v.
    SHKS = {v: sum(IS[u] for u in neighbors[v]) for v in G.nodes()}
    
    return SHKS



# method18: KSGC
import numpy as np
from heapq import heappop, heappush

def method18(G):
    # Precompute core numbers and degrees
    ks = nx.core_number(G)
    k = dict(G.degree())

    ks_max, ks_min = max(ks.values()), min(ks.values())
    denom = ks_max - ks_min if ks_max != ks_min else 1  # Avoid division by zero
    
    nodes = list(G.nodes())
    node_index = {node: i for i, node in enumerate(nodes)}

    # Convert to NumPy arrays
    ks_array = np.array([ks[node] for node in nodes], dtype=np.float64)
    k_array = np.array([k[node] for node in nodes], dtype=np.float64)
    
    n = len(nodes)
    
    # Compute shortest path lengths using Dijkstra (faster than Floyd-Warshall)
    shortest_paths = {v: nx.single_source_dijkstra_path_length(G, v) for v in G.nodes()}

    # Compute the neighborhood radius (half of the graph's diameter)
    diameter = nx.diameter(G)
    radius = int(0.5 * diameter)
    
    # Compute KSGC
    KSGC = {v: 0.0 for v in G.nodes()}
    
    for v in G.nodes():
        index_v = node_index[v]
        ks_v = ks_array[index_v]
        k_v = k_array[index_v]
        
        sum_F = 0.0
        for u, d in shortest_paths[v].items():
            if u == v or d > radius:
                continue  # Skip self-loops and nodes outside the radius
            
            index_u = node_index[u]
            ks_u = ks_array[index_u]
            k_u = k_array[index_u]
            
            # Compute C value
            C_val = np.exp((ks_v - ks_u) / denom)
            
            # Compute F value
            F_val = C_val * (k_v * k_u) / (d ** 2)
            
            sum_F += F_val
        
        KSGC[v] = sum_F

    return KSGC




def k_neighbors(G, node, radius):
        neighbors = set(nx.single_source_shortest_path_length(G, node, cutoff=radius).keys())
        neighbors.discard(node)  # Remove the node itself if present
        return neighbors
    
# Method 19 (InformationRank)
def method19(G):
    
    PROPA = {}
    Score = {}
    L = 2
    Miu = 0.2

    # Initialize PROPA and Score dictionaries
    for v in G.nodes():
        PROPA[v] = {}
        Score[v] = 0

    # Calculate PROPA and Score
    for v in G.nodes():
        neighbors = k_neighbors(G, v, L)
        for w in neighbors:
            PROPA[v][w] = 1
            path_length_counts = {}
            for path in nx.all_simple_paths(G, v, w, cutoff=L):
                length = len(path) - 1
                path_length_counts[length] = path_length_counts.get(length, 0) + 1

            for l in range(1, L + 1):
                pow_val = path_length_counts.get(l, 0)
                PROPA[v][w] *= (1 - Miu ** l) ** pow_val
            PROPA[v][w] = 1 - PROPA[v][w]
            Score[v] += PROPA[v][w]

    return Score

# method 20 (IS-PEW)
def method20(G):
    # Precompute expensive metrics
    triangles = nx.triangles(G)  # Dictionary {node: triangle_count}
    core_numbers = nx.core_number(G)  # Dictionary {node: core_number}
    degrees = dict(G.degree())  # Dictionary {node: degree}
    
    # Compute total triangles once
    TC = sum(triangles.values()) // 3 if sum(triangles.values()) > 0 else 1  # Avoid division by zero
    
    def compute_TP(v_A):
        return triangles[v_A] / TC
    
    def compute_InfE(v_A, v_B):
        common_neighbors = set(G[v_A]) & set(G[v_B])
        sum_DG_k = sum(degrees[k] for k in common_neighbors)
        return (degrees[v_A] * degrees[v_B]) / (1 + sum_DG_k)

    def compute_NIP(v_A):
        return sum(np.sqrt(degrees[v_a] * core_numbers[v_a]) for v_a in G[v_A])

    # Store edge weights in a dictionary to avoid recomputation
    edge_weights = {}

    def compute_EW(v_A, v_B):
        if (v_A, v_B) in edge_weights:
            return edge_weights[(v_A, v_B)]
        
        TP_v_A, TP_v_B = compute_TP(v_A), compute_TP(v_B)
        KS_v_A, KS_v_B = core_numbers[v_A], core_numbers[v_B]
        NIP_v_A, NIP_v_B = compute_NIP(v_A), compute_NIP(v_B)
        ED_v_A_v_B = 1 / compute_InfE(v_A, v_B)
        
        EW_value = ((KS_v_A * (1 + TP_v_A) * NIP_v_A) / ED_v_A_v_B) + ((KS_v_B * (1 + TP_v_B) * NIP_v_B) / ED_v_A_v_B)
        edge_weights[(v_A, v_B)] = EW_value  # Store result for future use
        edge_weights[(v_B, v_A)] = EW_value  # Store symmetric value
        return EW_value

    def compute_IS(v_A):
        return sum(compute_EW(v_A, v_B) for v_B in G[v_A])

    # Compute potential edge weights once
    for u, v in G.edges():
        G[u][v]['weight'] = compute_EW(u, v)

    # Compute IS values for all nodes
    IS_values = {v: compute_IS(v) for v in G.nodes()}
    
    return IS_values

# Method 21 (DKGM)

def method21(G):
    # Step 1: Efficient k-shell decomposition using batch removal
    G_copy = G.copy()
    k_shell = {}
    k_star = {}
    k = 1
    iteration = 1

    while G_copy.number_of_nodes() > 0:
        # Identify nodes to remove at current k-level
        nodes_to_remove = {node for node in G_copy.nodes if G_copy.degree[node] <= k}
        iteration_nodes = []
        
        while nodes_to_remove:
            iteration_nodes.extend([(node, iteration) for node in nodes_to_remove])
            G_copy.remove_nodes_from(nodes_to_remove)
            iteration += 1
            nodes_to_remove = {node for node in G_copy.nodes if G_copy.degree[node] <= k}

        # Assign k-shell and k-star values
        if iteration_nodes:
            m = iteration_nodes[-1][1]  # Last iteration number
            for node, n in iteration_nodes:
                k_shell[node] = k
                k_star[node] = k_shell[node] + (n / (m + 1))

        k += 1
        iteration = 1  # Reset for next k-shell

    # Step 2: Compute DK values
    DK = {node: G.degree[node] + k_star.get(node, 0) for node in G.nodes()}

    # Step 3: Precompute shortest path lengths (BFS for unweighted, Dijkstra for weighted)
    R = 2  # Neighborhood radius
    DKGM = {node: 0.0 for node in G.nodes()}

    for node in G.nodes():
        # Get shortest path lengths up to radius R using BFS
        sp_lengths = nx.single_source_shortest_path_length(G, node, cutoff=R)

        for neighbor, d_ij in sp_lengths.items():
            if node != neighbor and d_ij > 0:  # Avoid self-loops and division by zero
                DKGM[node] += DK[node] * DK.get(neighbor, 0) / (d_ij ** 2)

    return DKGM



#method 22: modularity_vitality (MV)
def method22(G):
    """Optimized version of method23 using Leiden algorithm for community detection."""
    
    def detect_communities_leiden(G):
        """Detect communities using Leiden algorithm efficiently."""
        # Convert NetworkX graph to iGraph.
        # Use to_numpy_array instead of the deprecated to_numpy_matrix.
        ig_G = ig.Graph.Adjacency((nx.to_numpy_array(G) > 0).tolist(), mode="undirected")
        ig_G.vs["name"] = list(G.nodes())
        partition = la.find_partition(ig_G, la.ModularityVertexPartition)
        return {ig_G.vs[i]["name"]: idx for idx, comm in enumerate(partition) for i in comm}

    communities = detect_communities_leiden(G)

    # Precompute degrees, edges per community, and modularity.
    M = G.number_of_edges()
    degree_cache = dict(G.degree())
    d_c = {c: sum(degree_cache[n] for n in G.nodes() if communities[n] == c)
           for c in set(communities.values())}

    def modularity_vitality(G, node, communities):
        """Compute the modularity vitality of a node using optimized calculations."""
        if not G.has_node(node):
            return 0

        c_i = communities[node]
        k_i = degree_cache[node]

        # Compute h_i,c values.
        hi_c = {c: sum(1 for neighbor in G.neighbors(node) if communities[neighbor] == c)
                for c in set(communities.values())}
        hi_c[c_i] += k_i  # Add k_i when node belongs to c_i.

        # Compute initial modularity.
        community_nodes = [n for n in G.nodes() if communities[n] == c_i]
        M_internal = sum(1 for n in community_nodes for neighbor in G.neighbors(n)
                         if communities[neighbor] == c_i) // 2
        k_i_internal = hi_c[c_i] - k_i  # Internal edges contribution.

        sum_term = sum((d_c[c] - hi_c[c]) ** 2 for c in hi_c)
        Q_updated = (M_internal - k_i_internal) / (M - k_i) - (1 / (4 * (M - k_i) ** 2)) * sum_term

        return Q_updated

    return {node: modularity_vitality(G, node, communities) for node in G.nodes()}


#method 23: Global-and-Local Centrality (GLC)
def method23(G, lam=1.0):
    """
    Compute the Global-and-Local Centrality (GLC) of each node in graph G.
    Optimized version with improved time complexity.
    
    Parameters:
        G   : networkx.Graph
              The input graph.
        lam : float (default=1.0)
              The fraction (lambda) of nodes that must be covered by clusters.
    
    Returns:
        GLC : dict
              A dictionary mapping each node in G to its GLC centrality value.
    """
    # Pre-compute and cache node degrees and neighbor sets
    degrees = dict(G.degree())
    neighbors = {node: set(G.neighbors(node)) for node in G.nodes()}
    total_nodes = G.number_of_nodes()
    
    # --- Step 1: Clustering and Global Critical Node Selection ---
    clusters = []
    assigned = set()
    
    # Helper function: Compute the potential of a node more efficiently
    def compute_potential(node):
        nbrs = neighbors[node]
        target_set = nbrs.union({node})
        s = 0
        for j in nbrs:
            # Use set intersection for faster computation
            s += len(neighbors[j].intersection(target_set))
        return degrees[node] * s

    # Pre-compute potentials for all nodes
    potentials = {node: compute_potential(node) for node in G.nodes()}
    
    while len(assigned) < total_nodes * lam:
        # Filter out assigned nodes from potentials
        available_potentials = {node: pot for node, pot in potentials.items() if node not in assigned}
        if not available_potentials:
            break
        
        # Select seed node with maximum potential
        seed = max(available_potentials, key=available_potentials.get)
        pcmax = available_potentials[seed]
        new_cluster = set([seed])
        assigned.add(seed)
        
        # Include neighbors of the seed with potential >= pcmax/2
        for neighbor in neighbors[seed]:
            if neighbor not in assigned and potentials[neighbor] >= pcmax / 2:
                new_cluster.add(neighbor)
                assigned.add(neighbor)
        
        # Expand the cluster iteratively (three degrees of influence)
        for _ in range(3):
            # More efficiently gather neighbors of the current cluster
            cluster_neighbors = set()
            for node in new_cluster:
                cluster_neighbors.update(neighbors[node] - new_cluster)
            
            # Pre-compute kin and kout for all candidates
            neighbor_metrics = {}
            for candidate in cluster_neighbors:
                if candidate in assigned:
                    continue
                # Use set operations for more efficient counting
                kin = len(neighbors[candidate].intersection(new_cluster))
                kout = degrees[candidate] - kin
                if kin >= kout:
                    neighbor_metrics[candidate] = (kin, degrees[candidate])
            
            # Sort by degree to process in order of increasing degree
            candidates = sorted(neighbor_metrics.keys(), key=lambda x: degrees[x])
            
            if not candidates:
                break
                
            # Add nodes that meet the criteria
            for candidate in candidates:
                new_cluster.add(candidate)
                assigned.add(candidate)
            
            if not candidates:  # No more nodes added
                break
        
        # Mark all neighbors of the new cluster as assigned
        for node in new_cluster:
            assigned.update(neighbors[node])
        
        clusters.append(new_cluster)
    
    # From each cluster, select the global critical node
    global_critical = [max(cluster, key=lambda node: degrees[node]) for cluster in clusters if cluster]
    
    # --- Step 2: Local Influence Calculation ---
    # Compute the k-shell (core) numbers in a single call
    core_numbers = nx.core_number(G)
    
    # More efficiently compute local influence
    LI = {}
    for node in G.nodes():
        LI[node] = sum(core_numbers[nbr] for nbr in neighbors[node])

    # --- Step 3: Overall GLC Centrality Calculation ---
    # Pre-compute all shortest paths for efficiency if graph is small enough
    # For large graphs, compute paths on-demand
    use_precomputed_paths = total_nodes <= 1000  # Threshold based on graph size
    
    if use_precomputed_paths:
        # Pre-compute shortest paths from all nodes to global critical nodes
        shortest_paths = {}
        for u in global_critical:
            paths = nx.single_source_shortest_path_length(G, u)
            for v, length in paths.items():
                shortest_paths[(v, u)] = length
    
    GLC = {}
    for v in G.nodes():
        global_factor = 0
        for u in global_critical:
            if use_precomputed_paths:
                try:
                    d = shortest_paths.get((v, u), float('inf'))
                except:
                    d = float('inf')
            else:
                try:
                    d = nx.shortest_path_length(G, source=v, target=u)
                except nx.NetworkXNoPath:
                    d = float('inf')
            d = max(d, 1)  # Avoid division by zero
            global_factor += LI[u] / (2 * d)
        GLC[v] = LI[v] * global_factor

    return GLC

# Method 24 (Weighted K-shell)
def method24(G):
    c1 = 0.1
    c2 = 0.4
    deg = dict(G.degree())
    ks = nx.core_number(G)
    ksdw = dict()

    for vi in G.nodes():
        ksdw[vi] = 0
        for vj in G.neighbors(vi):
            ksdw[vi] += (c1 * deg[vi] + c2 * ks[vi]) * (c1 * deg[vj] + c2 * ks[vj])

    return ksdw

# method 25:EFFD centrality
def method25(G):
    """
    Optimized version of the EffG centrality calculation.
    
    Parameters:
        G : networkx.Graph
            The input graph.
            
    Returns:
        effg_centrality : dict
            A dictionary mapping each node to its EffG centrality value.
    """
    import numpy as np
    import networkx as nx
    
    # Cache node degrees to avoid repeated lookups
    degree = dict(G.degree())
    nodes = list(G.nodes())
    n = len(nodes)
    
    # Initialize the result dictionary
    effg_centrality = {node: 0 for node in nodes}
    
    # Compute all shortest paths once using Dijkstra
    # This is more efficient than calling nx.all_pairs_dijkstra_path_length()
    # as it uses a more optimized implementation
    shortest_paths = dict(nx.all_pairs_dijkstra_path(G, weight=None))
    
    # Process each node pair once
    for i in range(n):
        node_i = nodes[i]
        k_i = degree[node_i]
        
        # Get all paths from node_i
        paths_from_i = shortest_paths[node_i]
        
        for j in range(i + 1, n):  # Only process each pair once (i,j) where i<j
            node_j = nodes[j]
            k_j = degree[node_j]
            
            # Get the path between i and j
            if node_j in paths_from_i:
                path = paths_from_i[node_j]
                path_length = len(path) - 1  # Number of edges in the path
                
                # Calculate effective distance
                if G.has_edge(node_i, node_j):
                    # Directly connected
                    p = 1 / degree[node_i]
                    d = 1 - np.log2(p) if p > 0 else float('inf')
                else:
                    # Indirectly connected
                    d = path_length
                
                # Calculate interaction score
                if d != float('inf'):
                    score = (k_i * k_j) / (d ** 2)
                    
                    # Update centrality values for both nodes
                    effg_centrality[node_i] += score
                    effg_centrality[node_j] += score
    
    return effg_centrality






#methods = {
#    "Degree Centrality": method1,
#    "Betweenness Centrality": method2,
#    "Closeness Centrality": method3,
#    "K-shell": method4,
#    "Cnc Plus": method5,
#    "Local H-index": method6,
#    "DNC": method7,
#    "ECLGC": method8,
#    "Centripetal Centrality": method9,
#    "EDBC" : method10,
#    "LGC" : method11,
#    "SEGM" : method12,
#    "MCGM" : method13,
#    "HVGC" : method14,
#    "BaseGM" : method15,
#    "HKS" : method16,
#    "SHKS" : method17,
#    "KSGC" : method18,
#    "Local Relative ASP" : method19,
#    "Information Rank" : method20,
#    "IS-PEW" : method21,
#    "DKGM" : method22,
#    "modularity vitality":method23,
#    "Global-and-Local Centrality (GLC)":method24,
#    "Weighted K-shell" : method25,
#    "EFFD gravity" : method26,
# }

# List of methods to be applied
methods = [globals()[f'method{i}'] for i in range(1, 26)]

dataset_folder = 'dataset'
beta_values = [0.02, 0.05, 0.08, 0.10, 0.13, 0.15, 0.18, 0.20, 0.22, 0.25, 0.30]
k_factors = [0.01, 0.03, 0.05, 0.07, 0.085, 0.10]
f_values = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1]

for dataset_filename in os.listdir(dataset_folder):
    if dataset_filename.endswith('.xlsx'):
        dataset_path = os.path.join(dataset_folder, dataset_filename)
        dataset_name = os.path.splitext(dataset_filename)[0]
        print(f"\nProcessing dataset: {dataset_name}...\n")
        
        df_edges = pd.read_excel(dataset_path, sheet_name='Sheet1')
        G = nx.from_pandas_edgelist(df_edges, source='source_column', target='target_column')
        G.remove_edges_from(nx.selfloop_edges(G))
        
        df_sir = pd.read_excel(dataset_path, sheet_name='SIR')
        df_sir.set_index('Node', inplace=True)
        
        num_nodes = len(G.nodes())
        results = {}
        method_results = {}

        for i, method in enumerate(methods, start=1):
            method_name = f'method{i}'
            print(f"Applying {method_name} on {dataset_name}...")
            try:
                ranked_nodes, exec_time = measure_execution_time(method, G)
                monotonicity = compute_monotonicity(G, ranked_nodes)
                method_results[method_name] = {
                    'ranked_nodes': ranked_nodes,
                    'execution_time': exec_time,
                    'monotonicity': monotonicity
                }
            except Exception as e:
                print(f"Error in {method_name} for {dataset_name}: {e}")
                method_results[method_name] = {
                    'ranked_nodes': [],
                    'execution_time': None,
                    'monotonicity': None,
                    'error': str(e)
                }

        for beta in beta_values:
            beta_column = f'Beta_{beta:.2f}'
            if beta_column not in df_sir.columns:
                print(f"Warning: {beta_column} not found in {dataset_name}. Skipping beta {beta}...")
                continue
            
            spread_power = df_sir[beta_column].to_dict()
            sigma = sorted(spread_power.keys(), key=lambda x: spread_power[x], reverse=True)
            
            for method_name, method_data in method_results.items():
                if 'error' in method_data:
                    results[(method_name, beta)] = {'Beta': beta, 'Kendall Tau': None, 'P-Value': None}
                    continue
                
                ranked_nodes = method_data['ranked_nodes']
                tau, p_value = kendalltau(
                    [spread_power[node] for node in sigma],
                    [spread_power[node] for node, _ in ranked_nodes]
                )
                
                R = [node for node, _ in ranked_nodes]
                jaccard_scores = {}
                si_scores = {}
                rbo_scores = {}
                mrr_scores = compute_mrr(sigma, R, k_factors)
                
                for k_factor in k_factors:
                    k = max(1, int(k_factor * num_nodes))
                    top_k_sigma = set(sigma[:k])
                    top_k_R = set(R[:k])
                    jaccard_scores[f'Jaccard k={k}'] = compute_jaccard_similarity(top_k_sigma, top_k_R)
                
                for f in f_values:
                    f_count = max(1, int(f * num_nodes))
                    top_f_nodes = [node for node, _ in ranked_nodes][:f_count]
                    si_scores[f'SI {f:.2f}'] = sum(spread_power[node] for node in top_f_nodes) / (f * num_nodes)
                    rbo_scores[f'RBO {f:.2f}'] = compute_rbo(sigma[:f_count], R[:f_count])
                
                results[(method_name, beta)] = {
                    'Beta': beta,
                    'Kendall Tau': tau,
                    'P-Value': p_value,
                    **jaccard_scores,
                    **si_scores,
                    **rbo_scores,
                    **mrr_scores
                }
        
        data = []
        for (method_name, beta), result in results.items():
            row = [
                dataset_name,  # Include dataset name in output
                method_name,
                method_results[method_name]['execution_time'],
                method_results[method_name].get('monotonicity', 'N/A'),
                result.get('Beta'),
                result.get('Kendall Tau'),
                result.get('P-Value')
            ] + [result.get(f'Jaccard k={int(k_factor * num_nodes)}') for k_factor in k_factors] + \
              [result.get(f'SI {f:.2f}') for f in f_values] + \
              [result.get(f'RBO {f:.2f}') for f in f_values] + \
              [result.get(f'MRR {k_factor:.2f}') for k_factor in k_factors]
            
            data.append(row)
        
        expected_columns = ['Dataset', 'Method', 'Execution Time', 'Monotonicity', 'Beta', 'Kendall Tau', 'P-Value'] + \
                           [f'Jaccard k={int(k_factor * num_nodes)}' for k_factor in k_factors] + \
                           [f'SI {f:.2f}' for f in f_values] + \
                           [f'RBO {f:.2f}' for f in f_values] + \
                           [f'MRR {k_factor:.2f}' for k_factor in k_factors]
        
        result_df = pd.DataFrame(data, columns=expected_columns)
        
        output_file = f'{dataset_name}_results.xlsx'
        with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
            df_edges.to_excel(writer, sheet_name='Sheet1', index=False)
            df_sir.to_excel(writer, sheet_name='SIR')
            result_df.to_excel(writer, sheet_name='Results', index=False)
        
        print(f"✅ Completed processing: {dataset_name}. Results saved in {output_file}\n")


Processing dataset: PowerGrid(SIR)...

Applying method1 on PowerGrid(SIR)...
Applying method2 on PowerGrid(SIR)...
Applying method3 on PowerGrid(SIR)...
Applying method4 on PowerGrid(SIR)...
Applying method5 on PowerGrid(SIR)...
Applying method6 on PowerGrid(SIR)...
Applying method7 on PowerGrid(SIR)...
Applying method8 on PowerGrid(SIR)...
Applying method9 on PowerGrid(SIR)...
Applying method10 on PowerGrid(SIR)...
Applying method11 on PowerGrid(SIR)...
Applying method12 on PowerGrid(SIR)...
Applying method13 on PowerGrid(SIR)...
Applying method14 on PowerGrid(SIR)...
Applying method15 on PowerGrid(SIR)...
Applying method16 on PowerGrid(SIR)...
Applying method17 on PowerGrid(SIR)...
Applying method18 on PowerGrid(SIR)...
Applying method19 on PowerGrid(SIR)...
Applying method20 on PowerGrid(SIR)...
Applying method21 on PowerGrid(SIR)...
Applying method22 on PowerGrid(SIR)...
Applying method23 on PowerGrid(SIR)...
Applying method24 on PowerGrid(SIR)...
Applying method25 on PowerGrid(SI

KeyError: 149